In [1]:
# import necessary libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# Will be using api to retrieve info
url = "https://www.coffeebean.com.my/amlocator/index/ajax/"
response = requests.get(url)
stores = response.json()

In [3]:
# Extracting information

# Expected response from the api
#{
#    "items": [
#        {
#            "name": "Coffee Bean Kuala Lumpur",
#            "lat": 3.14159,
#            "lng": 101.12345,
#            "popup_html": '<div class="amlocator-info-popup"> <h3 class="amlocator-name"><div class="amlocator-title"><a class="amlocator-link" href="https://www.coffeebean.com.my/amlocator/all seasons place/" title="All Seasons Place" target="_blank">All Seasons Place</a></div></h3> <div class="amlocator-image"></div> Unit No: 6G-1-12 , Level 1 All Seasons Place, Lebuhraya Thean Teik Bandar Baru Air Itam Penang 11500 <div class="amlocator-description"><div data-content-type="row" data-appearance="contained" data-element="main"><div data-enable-parallax="0" data-parallax-speed="0.5" data-background-images="{}" data-background-type="image" data-video-loop="true" data-video-play-only-visible="true" data-video-lazy-load="true" data-video-fallback-src="" data-element="inner" style="justify-content: flex-start; display: flex; flex-direction: column; background-position: left top; background-size: cover; background-repeat: no-repeat; background-attachment: scroll; border-style: none; border-width: 1px; border-radius: 0px; margin: 0px 0px 10px; padding: 10px;"><div data-content-type="text" data-appearance="default" data-element="main" style="border-style: none; border-width: 1px; border-radius: 0px; margin: 0px; padding: 0px;"><p><strong>Opening Hours</strong></p> <p>Sun - Thu: 8.00 AM – 10.00 PM<br>Fri,Sat, &amp; Eve of PH: 8.00 AM - 11.00 PM</p></div></div></div></div> </div>'},
#        },
#    ]
#}

data = []
for store in stores['items']:
    name = store.get("name")
    lat = store.get("lat")
    lng = store.get("lng")
    popup_html = store.get("popup_html")

    # Parse the popup HTML
    soup = BeautifulSoup(popup_html, 'html.parser')

    address_tag = soup.select_one(".amlocator-image")
    address = address_tag.next_sibling.strip() if address_tag else ""

    hours = ""
    strong_tag = soup.find("strong", string="Opening Hours")
    if strong_tag:
        p_tag = strong_tag.find_parent("p")
        if p_tag:
            next_p_tag = p_tag.find_next_sibling("p")
            if next_p_tag:
                hours = next_p_tag.get_text(strip=True).replace("\n", "")

    data.append({
        "name": name,
        "latitude": lat,
        "longitude": lng,
        "address": address,
        "opening_hours": hours
    })

# Once already extracted, we can convert to dataframe
df = pd.DataFrame(data)
df

,name,latitude,longitude,address,opening_hours
0,All Seasons Place,5.39634250,100.29049810,"Unit No: 6G-1-12 , Level 1 All Seasons Place, ...","Sun - Thu: 8.00 AM – 10.00 PMFri,Sat, & Eve of..."
1,Ativo Plaza,3.19536350,101.61687390,"Level 1, Ativo Plaza, Lot A-G-2, Block A, No. ...",Daily: 7.00 AM - 10.30 PM
2,Atria Shopping Gallery,3.12709630,101.61651760,"Lot G38, Ground Floor, Atria Shopping Gallery,...",Daily: 10.00 AM - 10.00 PM
3,Botanica Balik Pulau,5.37248700,100.21967500,"142A, Ground Floor, Jalan Sungai Air Putih Bal...",Daily: 9.30 AM - 9.30 PM
4,Bagan Ajam Drive Thru,5.43887370,100.38309700,"No.9, Lorong Oren, Pusat Perniagaan Oren Butte...",Daily: 7.00 AM - 12.30 AM
...,...,...,...,...,...
192,AEON Kota Bharu,6.11644920,102.22826060,"Lot G37, Ground Floor, AEON Mall Kota Bharu, K...",
193,Ayer Keroh Melaka,2.25335750,102.29068480,"No 1, Ground Floor, Jalan Komersial Takh 2, Ta...",Daily: 8.00 AM - 12.00 AM
194,Ampang Point,3.15917070,101.75128240,"No.51, Jalan Mamanda 9 Ampang Point Ampang Sel...",
195,Plaza Arkadia,3.18620540,101.63613640,"A-G-11 (Block A), Plaza Arkadia, No. 3, Jalan ...",


In [4]:
# Cleaning Data for Opening Hours
# There are some value that need to be cleaned such as PMFri, PMSat, PMSun, PMOrder, 7.00AM - 10.30PM
# Using a simple replace to clean this section

df['opening_hours'] = df['opening_hours'].str.replace("PMFri", "PM | Fri").str.strip()
df['opening_hours'] = df['opening_hours'].str.replace("PMSat", "PM | Sat").str.strip()
df['opening_hours'] = df['opening_hours'].str.replace("PMSun", "PM | Sun").str.strip()
df['opening_hours'] = df['opening_hours'].str.replace("PMOrder", "PM | Order").str.strip()
df['opening_hours'] = df['opening_hours'].str.replace("7.00AM - 10.30PM", "7.00 AM - 10.30 PM").str.strip()
df['opening_hours'].unique().tolist()

['Sun - Thu: 8.00 AM – 10.00 PM | Fri,Sat, & Eve of PH: 8.00 AM - 11.00 PM',
 'Daily: 7.00 AM - 10.30 PM',
 'Daily: 10.00 AM - 10.00 PM',
 'Daily: 9.30 AM - 9.30 PM',
 'Daily: 7.00 AM - 12.30 AM',
 'Daily: 7.30 AM - 11.30 PM',
 'Daily: 7.30 AM - 10.00 PM',
 'Daily: 7.30 AM - 11.00 PM',
 'Daily: 8.30 AM - 10.30 PM',
 'Daily: 9.00 AM - 10.00 PM',
 'Daily: 8.00 AM - 10.00 PM',
 '',
 'Daily: 9.30 AM - 10.00 PM',
 'Daily: 7.00 AM - 8.00 PM',
 'Daily: 7.00 AM - 11.00 PM',
 'Daily: 8.30 AM - 10.00 PM',
 'Daily: 8.00 AM - 10.30 PM',
 'Daily: 7.00 AM - 8.30 PM',
 'Daily: 8.00 AM - 11.00 PM',
 'Sun - Thu: 9.00 AM - 1.00 PM | Fri, Sat & PH: 9.00 AM - 10:30 PM',
 'Daily: 10.00 AM - 9.00 PM',
 'Sun - Thu: 10.00 AM – 10:00 PM | Fri,Sat, & PH: 10.00 AM - 10.30 PM',
 'Daily: 7.30 AM - 10.30 PM',
 'Daily: 7.00 AM - 10.00 PM',
 'Daily: 7.30 AM - 12.00 AM',
 'Daily: 9.30 AM - 10.30 PM',
 'Daily: 7.00 AM - 11.30 PM',
 'Sun - Thu: 10.00 AM - 10.00 PM',
 'Daily: 7.00 AM - 12.00 AM',
 'Sun - Thur: 7.30 AM - 

In [5]:
# Cleaning Data for Opening Hours
# Capitalize the AM and PM
df['opening_hours'] = df['opening_hours'].str.replace("am", "AM").str.strip()
df['opening_hours'] = df['opening_hours'].str.replace("pm", "PM").str.strip()
df['opening_hours'].unique().tolist()

['Sun - Thu: 8.00 AM – 10.00 PM | Fri,Sat, & Eve of PH: 8.00 AM - 11.00 PM',
 'Daily: 7.00 AM - 10.30 PM',
 'Daily: 10.00 AM - 10.00 PM',
 'Daily: 9.30 AM - 9.30 PM',
 'Daily: 7.00 AM - 12.30 AM',
 'Daily: 7.30 AM - 11.30 PM',
 'Daily: 7.30 AM - 10.00 PM',
 'Daily: 7.30 AM - 11.00 PM',
 'Daily: 8.30 AM - 10.30 PM',
 'Daily: 9.00 AM - 10.00 PM',
 'Daily: 8.00 AM - 10.00 PM',
 '',
 'Daily: 9.30 AM - 10.00 PM',
 'Daily: 7.00 AM - 8.00 PM',
 'Daily: 7.00 AM - 11.00 PM',
 'Daily: 8.30 AM - 10.00 PM',
 'Daily: 8.00 AM - 10.30 PM',
 'Daily: 7.00 AM - 8.30 PM',
 'Daily: 8.00 AM - 11.00 PM',
 'Sun - Thu: 9.00 AM - 1.00 PM | Fri, Sat & PH: 9.00 AM - 10:30 PM',
 'Daily: 10.00 AM - 9.00 PM',
 'Sun - Thu: 10.00 AM – 10:00 PM | Fri,Sat, & PH: 10.00 AM - 10.30 PM',
 'Daily: 7.30 AM - 10.30 PM',
 'Daily: 7.00 AM - 10.00 PM',
 'Daily: 7.30 AM - 12.00 AM',
 'Daily: 9.30 AM - 10.30 PM',
 'Daily: 7.00 AM - 11.30 PM',
 'Sun - Thu: 10.00 AM - 10.00 PM',
 'Daily: 7.00 AM - 12.00 AM',
 'Sun - Thur: 7.30 AM - 

In [6]:
# Cleaning Data for Opening Hours
# This block will convert the column Opening Hours into separate column for each day and convert it into 24-hour format

from datetime import datetime
import re


def convert_time(time: str):
    """
    Helper to convert time to 24-hour format
    :param time:
        - time in 12-hour format
    :return:
        - time in 24-hour format
        - None if time is not in 12-hour format
    """
    try:
        return datetime.strptime(time.strip().lower().replace('.', ':'), "%I:%M %p").strftime("%H:%M")
    except:
        return None


# List of day keys including Public Holiday (PH)
DAY_KEYS = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', 'PH']

# Mapping from short day strings to full names
DAY_MAP = {
    "Mon": "Monday",
    "Tue": "Tuesday",
    "Wed": "Wednesday",
    "Thu": "Thursday",
    "Thur": "Thursday",
    "Fri": "Friday",
    "Sat": "Saturday",
    "Sun": "Sunday",
    "PH": "PH",
    "Public Holiday": "PH",
    "Eve of PH": "PH"
}


def expand_days(day_range: str):
    """
    Helper to expand day range to list of days
    :param day_range:
        - day range in format 'Sun - Thu'
    :return:
        - list of days in day_range
    """

    day_order = ['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']
    start, end = day_range.split('-')
    start = start.strip()
    end = end.strip()
    start_idx = day_order.index(start)
    end_idx = day_order.index(end)
    if start_idx <= end_idx:
        return day_order[start_idx:end_idx + 1]
    else:
        return day_order[start_idx:] + day_order[:end_idx + 1]


def parse_opening_hours(row: str):
    """
    Helper to parse opening hours into dictionary
    :param row:
        - opening hours in format 'Sun - Thu: 8.00 AM - 10.00 PM' or 'Daily: 8.00 AM - 10.00 PM'
    :return:
        - dictionary of opening hours for each day
    """
    result = {day: None for day in DAY_KEYS}
    text = row or ""
    parts = re.split(r'\s*\|\s*', text)  # split by '|'
    for part in parts:
        days = []
        # Regex to find time range based on AM/PM
        time_match = re.findall(r"(\d{1,2}[.:]\d{2}\s*[APap][Mm])\s*[-–]\s*(\d{1,2}[.:]\d{2}\s*[APap][Mm])", part)
        if not time_match:
            continue
        open_time, close_time = convert_time(time_match[0][0]), convert_time(time_match[0][1])
        time_str = f"{open_time}-{close_time}"

        # Find days
        if "daily" in part.lower():
            days = DAY_KEYS  # All including PH
        else:
            for short, full in DAY_MAP.items():
                if re.search(rf'\b{short}\b', part, re.IGNORECASE):
                    days.append(full)
            # Regex to find day ranges like 'Sun - Thu'
            range_match = re.findall(r"(Sun|Mon|Tue|Wed|Thu|Thur|Fri|Sat)\s*[-–]\s*(Sun|Mon|Tue|Wed|Thu|Thur|Fri|Sat)",
                                     part, re.IGNORECASE)
            for r in range_match:
                days += [DAY_MAP[d] for d in expand_days(f"{r[0]}-{r[1]}")]

        # Remove duplicates if any
        days = list(set(days))
        for day in days:
            result[day] = time_str
    return pd.Series(result)


# Apply the function and split the result into separate columns based on opening hours
df[['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday', 'public_holiday']] = df[
    'opening_hours'].apply(parse_opening_hours)
df

,name,latitude,longitude,address,opening_hours,monday,tuesday,wednesday,thursday,friday,saturday,sunday,public_holiday
0,All Seasons Place,5.39634250,100.29049810,"Unit No: 6G-1-12 , Level 1 All Seasons Place, ...","Sun - Thu: 8.00 AM – 10.00 PM | Fri,Sat, & Eve...",08:00-22:00,08:00-22:00,08:00-22:00,08:00-22:00,08:00-23:00,08:00-23:00,08:00-22:00,08:00-23:00
1,Ativo Plaza,3.19536350,101.61687390,"Level 1, Ativo Plaza, Lot A-G-2, Block A, No. ...",Daily: 7.00 AM - 10.30 PM,07:00-22:30,07:00-22:30,07:00-22:30,07:00-22:30,07:00-22:30,07:00-22:30,07:00-22:30,07:00-22:30
2,Atria Shopping Gallery,3.12709630,101.61651760,"Lot G38, Ground Floor, Atria Shopping Gallery,...",Daily: 10.00 AM - 10.00 PM,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00
3,Botanica Balik Pulau,5.37248700,100.21967500,"142A, Ground Floor, Jalan Sungai Air Putih Bal...",Daily: 9.30 AM - 9.30 PM,09:30-21:30,09:30-21:30,09:30-21:30,09:30-21:30,09:30-21:30,09:30-21:30,09:30-21:30,09:30-21:30
4,Bagan Ajam Drive Thru,5.43887370,100.38309700,"No.9, Lorong Oren, Pusat Perniagaan Oren Butte...",Daily: 7.00 AM - 12.30 AM,07:00-00:30,07:00-00:30,07:00-00:30,07:00-00:30,07:00-00:30,07:00-00:30,07:00-00:30,07:00-00:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,AEON Kota Bharu,6.11644920,102.22826060,"Lot G37, Ground Floor, AEON Mall Kota Bharu, K...",,None,None,None,None,None,None,None,None
193,Ayer Keroh Melaka,2.25335750,102.29068480,"No 1, Ground Floor, Jalan Komersial Takh 2, Ta...",Daily: 8.00 AM - 12.00 AM,08:00-00:00,08:00-00:00,08:00-00:00,08:00-00:00,08:00-00:00,08:00-00:00,08:00-00:00,08:00-00:00
194,Ampang Point,3.15917070,101.75128240,"No.51, Jalan Mamanda 9 Ampang Point Ampang Sel...",,None,None,None,None,None,None,None,None
195,Plaza Arkadia,3.18620540,101.63613640,"A-G-11 (Block A), Plaza Arkadia, No. 3, Jalan ...",,None,None,None,None,None,None,None,None


In [7]:
# Rename the column for better understanding
df = df.rename(columns={"opening_hours": "raw_opening_hours"})

In [8]:
# Cleaning the data of latitude and longitude
# Removing the 0.00000000 if any
df['latitude'] = df['latitude'].str.replace("0.00000000", "").str.strip()
df['longitude'] = df['longitude'].str.replace("0.00000000", "").str.strip()
df['longitude'].unique().tolist()

['100.29049810',
 '101.61687390',
 '101.61651760',
 '100.21967500',
 '100.38309700',
 '102.95416320',
 '101.66736530',
 '101.67074170',
 '101.38850710',
 '101.74211750',
 '101.61140500',
 '101.95531330',
 '101.90518730',
 '100.54673760',
 '103.76337400',
 '101.59255090',
 '101.62753500',
 '101.62712650',
 '101.65096800',
 '103.62026340',
 '100.31532340',
 '101.73946140',
 '101.58280380',
 '100.32004500',
 '103.87170060',
 '100.31057470',
 '100.47846160',
 '101.53351620',
 '101.78311590',
 '100.27518750',
 '100.31931370',
 '101.65681370',
 '100.43449380',
 '101.71367780',
 '101.61808820',
 '101.11865270',
 '101.76029010',
 '102.24585110',
 '103.62148250',
 '101.07149250',
 '101.63570200',
 '101.63728830',
 '103.32772160',
 '101.71208990',
 '101.72980590',
 '103.32057980',
 '103.76322900',
 '100.32858810',
 '103.76288110',
 '101.67890880',
 '101.71162190',
 '101.67689640',
 '102.28140290',
 '103.44825130',
 '100.31236500',
 '101.77483280',
 '101.65302640',
 '101.67805450',
 '101.68703470

In [9]:
# Cleaning the data to extract postcode and state from address column

import re

STATES = [
    "Johor", "Kedah", "Kelantan", "Malacca", "Negeri Sembilan", "Pahang",
    "Penang", "Perak", "Perlis", "Sabah", "Sarawak", "Selangor",
    "Terengganu", "Kuala Lumpur", "Putrajaya", "Labuan", "Federal Territories"
]


def extract_address_info(address: str):
    """
    Extract postcode and state from address
    :param address: string of address
    :return: dict: extracted info from address based on this format
        {
            'street_info': street_info,
            'postcode': postcode,
            'state': state
        }
    """
    postcodes = re.findall(r'\b\d{5}\b', address)
    postcode = postcodes[-1] if postcodes else None

    # Define state mappings
    WILAYAH_MAP = {
        'Kuala Lumpur': 'Wilayah Persekutuan Kuala Lumpur',
        'Putrajaya': 'Wilayah Persekutuan Putrajaya',
        'Labuan': 'Wilayah Persekutuan Labuan',
    }

    # Check if there is a match of any of the states in the address
    found_state = next((s for s in WILAYAH_MAP.keys() if s in address), None)
    if not found_state:
        found_state = next((s for s in STATES if s in address), None)

    # Replace with full wilayah label if matched
    if found_state in WILAYAH_MAP:
        found_state = WILAYAH_MAP[found_state]

    # Improved city extraction
    city = None
    if postcode:
        postcode_idx = address.rfind(postcode)
        before_postcode = address[:postcode_idx]

        # Known cities, to be checked first. Can be extended
        KNOWN_CITIES = ['Kuala Lumpur', 'Ipoh', 'George Town', 'Petaling Jaya', 'Shah Alam',
                        'Seremban', 'Johor Bahru', 'Kota Bharu', 'Bukit Mertajam']
        for c in KNOWN_CITIES:
            if c in before_postcode:
                city = c
                break

        # If there is no known cities, will extracting the last 1–2 capitalized words
        if not city:
            tokens = re.split(r',|\n', before_postcode)
            last_token = tokens[-1].strip()
            words = last_token.split()
            if len(words) >= 2 and words[-2].lower() == 'kuala' and words[-1].lower() == 'lumpur':
                city = 'Kuala Lumpur'
            elif len(words) >= 1:
                city = ' '.join(words[-2:]) if len(words) >= 2 else words[-1]

    # Clean remaining as street info, will be including city as part of it
    cleaned = address
    if postcode:
        cleaned = cleaned.replace(postcode, '')
    if found_state:
        cleaned = cleaned.replace(found_state, '')
    if city:
        cleaned = cleaned.replace(city, '')
    street_info = f"{city if city else ''}, {cleaned.strip(' ,.')}".strip(', ')

    return pd.Series({
        'street_info': street_info,
        'postcode': postcode,
        'state': found_state
    })


# Apply the function and split the result into separate columns from address column
df[['street_info', 'postcode', 'state']] = df['address'].apply(extract_address_info)
df


,name,latitude,longitude,address,raw_opening_hours,monday,tuesday,wednesday,thursday,friday,saturday,sunday,public_holiday,street_info,postcode,state
0,All Seasons Place,5.39634250,100.29049810,"Unit No: 6G-1-12 , Level 1 All Seasons Place, ...","Sun - Thu: 8.00 AM – 10.00 PM | Fri,Sat, & Eve...",08:00-22:00,08:00-22:00,08:00-22:00,08:00-22:00,08:00-23:00,08:00-23:00,08:00-22:00,08:00-23:00,"Itam Penang, Unit No: 6G-1-12 , Level 1 All Se...",11500,Penang
1,Ativo Plaza,3.19536350,101.61687390,"Level 1, Ativo Plaza, Lot A-G-2, Block A, No. ...",Daily: 7.00 AM - 10.30 PM,07:00-22:30,07:00-22:30,07:00-22:30,07:00-22:30,07:00-22:30,07:00-22:30,07:00-22:30,07:00-22:30,"Kuala Lumpur, Level 1, Ativo Plaza, Lot A-G-2,...",52200,Wilayah Persekutuan Kuala Lumpur
2,Atria Shopping Gallery,3.12709630,101.61651760,"Lot G38, Ground Floor, Atria Shopping Gallery,...",Daily: 10.00 AM - 10.00 PM,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,"Petaling Jaya, Lot G38, Ground Floor, Atria Sh...",47400,Selangor
3,Botanica Balik Pulau,5.37248700,100.21967500,"142A, Ground Floor, Jalan Sungai Air Putih Bal...",Daily: 9.30 AM - 9.30 PM,09:30-21:30,09:30-21:30,09:30-21:30,09:30-21:30,09:30-21:30,09:30-21:30,09:30-21:30,09:30-21:30,"Pulau Penang, 142A, Ground Floor, Jalan Sungai...",11000,Penang
4,Bagan Ajam Drive Thru,5.43887370,100.38309700,"No.9, Lorong Oren, Pusat Perniagaan Oren Butte...",Daily: 7.00 AM - 12.30 AM,07:00-00:30,07:00-00:30,07:00-00:30,07:00-00:30,07:00-00:30,07:00-00:30,07:00-00:30,07:00-00:30,"Butterworth Penang, No.9, Lorong Oren, Pusat P...",13000,Penang
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,AEON Kota Bharu,6.11644920,102.22826060,"Lot G37, Ground Floor, AEON Mall Kota Bharu, K...",,None,None,None,None,None,None,None,None,"Kota Bharu, Lot G37, Ground Floor, AEON Mall ,...",15050,Kelantan
193,Ayer Keroh Melaka,2.25335750,102.29068480,"No 1, Ground Floor, Jalan Komersial Takh 2, Ta...",Daily: 8.00 AM - 12.00 AM,08:00-00:00,08:00-00:00,08:00-00:00,08:00-00:00,08:00-00:00,08:00-00:00,08:00-00:00,08:00-00:00,"Malacca Malacca, No 1, Ground Floor, Jalan Kom...",75450,Malacca
194,Ampang Point,3.15917070,101.75128240,"No.51, Jalan Mamanda 9 Ampang Point Ampang Sel...",,None,None,None,None,None,None,None,None,"Ampang Selangor, No.51, Jalan Mamanda 9 Ampang...",68000,Selangor
195,Plaza Arkadia,3.18620540,101.63613640,"A-G-11 (Block A), Plaza Arkadia, No. 3, Jalan ...",,None,None,None,None,None,None,None,None,"Kuala Lumpur, A-G-11 (Block A), Plaza Arkadia,...",52200,Wilayah Persekutuan Kuala Lumpur


In [10]:
# Rename the column for better understanding
df = df.rename(columns={"address": "raw_address"})

In [13]:
# Order the columns
df = df[
    ['name', 'street_info', 'postcode', 'state', 'raw_address', 'latitude', 'longitude', 'raw_opening_hours', 'monday',
     'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday', 'public_holiday']]

In [14]:
# Export to excel or CSV
# Must change the path, make sure the file name is .xlsx

USER_PATH = "/Users/adibrafi/Desktop/"
FILE_NAME = "cbtlLocation.xlsx"

df.to_excel(USER_PATH + FILE_NAME, index=False)
